In [1]:
  #!/usr/bin/env python

try:
    import gi
    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as nc
from gi.repository import NumCosmoMath as ncm
import numpy as np
from astropy.io import fits
from astropy.table import Table
import sys
sys.path.insert(0,'../../scripts')

import pyccl as ccl

from nc_ccl import create_nc_obj, ccl_cosmo_set_high_prec

ncm.cfg_init()
ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

initialize the `Cosmological models`

In [2]:
#CCL cosmology
cosmo_ccl = ccl.Cosmology(Omega_c = 0.30711 - 0.048254, Omega_b = 0.048254, h = 0.677, sigma8 = 0.8822714165197718, n_s=0.96, Omega_k = 0, transfer_function='eisenstein_hu')
#ccl_cosmo_set_high_prec (cosmo_ccl)

cosmo_numcosmo, dist, ps_lin, ps_nln, hmfunc = create_nc_obj (cosmo_ccl)

psf = hmfunc.peek_psf ()

# Define proxy modelling
Use a mass proxy, define the probability for observing a proxy given a mass and redhsift

$$
P(\log\lambda|M,z) = N(\mu(M,z), \sigma^2(M,z))
$$
the mean is
$$
\mu(M,z) = \mu_0 + a_\mu^M\log_{10}\frac{M}{M_0} + a_\mu^z\log_{10}\frac{1+z}{1+z_0} 
$$
variance is
$$
\sigma(M,z) = \sigma_0 + a_\sigma^M\log_{10}\frac{M}{M_0} + a_\sigma ^z\log_{10}\frac{1+z}{1+z_0} 
$$

In [3]:
#CosmoSim_proxy model
#M_0, z_0
theta_pivot = [3e14/0.71, 0.6]
#\mu_0, a_\mu^z, a_\mu^M
theta_mu = [3.19, -0.7, 2]
#\sigma_0, a_\sigma^z, a_\sigma^M
theta_sigma = [0.33, 0.,-0.08]
#Richness object

area = (0.25)*4*np.pi / 100.0
lnRl = 1.0
lnRu = 2.0
zl = 0.25
zu = 1.0

#Numcosmo_proxy model
cluster_z = nc.ClusterRedshift.new_from_name("NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (zl, zu))
cluster_m = nc.ClusterMass.new_from_name("NcClusterMassAscaso{'M0':<%20.15e>,'z0':<%20.15e>,'lnRichness-min':<%20.15e>, 'lnRichness-max':<%20.15e>}" % (3e14/(0.71),0.6, lnRl, lnRu))
cluster_m.param_set_by_name('mup0', 3.19)
cluster_m.param_set_by_name('mup1', 2/np.log(10))
cluster_m.param_set_by_name('mup2', -0.7/np.log(10))
cluster_m.param_set_by_name('sigmap0', 0.33)
cluster_m.param_set_by_name('sigmap1', -0.08/np.log(10))
cluster_m.param_set_by_name('sigmap2', 0/np.log(10))

initialize the `ClusterAbundance` object

In [4]:
#Numcosmo Cluster Abundance

#First we need to define the multiplicity function here we will use the tinker
mulf = nc.MultiplicityFuncTinker.new()
mulf.set_linear_interp (True)
mulf.set_mdef(nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200)
#Second we need to construct a filtered power spectrum 

hmf = nc.HaloMassFunction.new(dist,psf,mulf)
hmf.set_area(area)

ca = nc.ClusterAbundance.new(hmf,None)
mset = ncm.MSet.new_array([cosmo_numcosmo,cluster_m,cluster_z])

In [5]:
ncount = Nc.DataClusterNCount.new (ca, "NcClusterRedshiftNodist", "NcClusterMassAscaso")
ncount.catalog_load ("ncount_ascaso.fits")

In [6]:
cosmo_numcosmo.props.Omegac_fit = True
cosmo_numcosmo.props.w0_fit = True
cluster_m.props.mup0_fit = True
mset.prepare_fparam_map ()

In [7]:
ncount.set_binned (True)

In [8]:
dset = ncm.Dataset.new ()
dset.append_data (ncount)

In [9]:
lh = Ncm.Likelihood (dataset = dset)
fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_FORWARD)

In [10]:
fitmc = Ncm.FitMC.new (fit, Ncm.FitMCResampleType.FROM_MODEL, Ncm.FitRunMsgs.FULL)
fitmc.set_nthreads (3)

fitmc.set_data_file ("ncount_ascaso_mc_binned.fits")

fitmc.start_run ()
fitmc.run_lre (50, 1.0e-2)
fitmc.end_run ()

#----------------------------------------------------------------------------------
# NcmFitMC: Starting Monte Carlo...
#----------------------------------------------------------------------------------
# Data used:
#   - Cluster abundance unbinned
#----------------------------------------------------------------------------------
# NcmFitMC: Fiducial model set:
#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : Chevalier-Polarski-Linder parametrization
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  67.7                [FIXED]
#   -     Omegac[01]:  0.258856            [FREE]
#   -     Omegak[02]:  0                   [FIXED]
#   -    Tgamma0[03]:  2.725               [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.048254            [FIXED]
#   -         w

#  m2lnL     =  1.64639767639453e-10 (  1.6463977e-10 )
#  Fit parameters:
#     0.25823025519261    -0.991579498701987    3.18665858008232    
# NcmMSetCatalog: Current mean:   1.0914e-07   0.259       -0.9933       3.1887     
# NcmMSetCatalog: Current msd:    6.2962e-08   0.00038667   0.0010742    0.0024959  
# NcmMSetCatalog: Current sd:     1.2592e-07   0.00077334   0.0021484    0.0049919  
# NcmMSetCatalog: Current var:    1.5857e-14   5.9805e-07   4.6155e-06   2.4919e-05 
# NcmMSetCatalog: Current tau:    1            1            1            1          
# Task:NcmFitMC, completed: 4 of 50, elapsed time: 00:01:26.8192
# Task:NcmFitMC, mean time: 00:00:21.6803 +/- 00:00:15.3174
# Task:NcmFitMC, time left: 00:16:37.2936 +/- 00:11:44.6024
# Task:NcmFitMC, current time:        Wed Apr 06 2022, 00:34:03
# Task:NcmFitMC, estimated to end at: Wed Apr 06 2022, 00:50:40 +/- 00:11:44.6024
#  Elapsed time: 00 days, 00:00:58.6696390
#  iteration            [000068]
#  function evaluations 

# Task:NcmFitMC, time left: 00:08:59.7299 +/- 00:04:43.8776
# Task:NcmFitMC, current time:        Wed Apr 06 2022, 00:35:08
# Task:NcmFitMC, estimated to end at: Wed Apr 06 2022, 00:44:08 +/- 00:04:43.8776
#  Elapsed time: 00 days, 00:01:27.5655360
#  iteration            [000073]
#  function evaluations [000075]
#  gradient evaluations [000000]
#  degrees of freedom   [012135]
#  m2lnL     =  6.56320080782979e-08 (  6.5632008e-08 )
#  Fit parameters:
#     0.258744358203331   -0.996863505160227    3.19272567308917    
# NcmMSetCatalog: Current mean:   3.7889e-07   0.25965     -0.99563      3.1819     
# NcmMSetCatalog: Current msd:    1.2629e-07   0.00027717   0.0006408    0.0059899  
# NcmMSetCatalog: Current sd:     4.3749e-07   0.00096015   0.0022198    0.020749   
# NcmMSetCatalog: Current var:    1.914e-13    9.2189e-07   4.9275e-06   0.00043054 
# NcmMSetCatalog: Current tau:    1            1            1            1          
# Task:NcmFitMC, completed: 12 of 50, elapsed time

# NcmMSetCatalog: Current msd:    2.0269e-07   0.00021818   0.00044997   0.0046551  
# NcmMSetCatalog: Current sd:     8.835e-07    0.00095101   0.0019614    0.020291   
# NcmMSetCatalog: Current var:    7.8057e-13   9.0442e-07   3.847e-06    0.00041173 
# NcmMSetCatalog: Current tau:    1            1            1            1          
# Task:NcmFitMC, completed: 19 of 50, elapsed time: 00:03:50.8270
# Task:NcmFitMC, mean time: 00:00:12.1423 +/- 00:00:04.6332
# Task:NcmFitMC, time left: 00:06:16.4117 +/- 00:02:23.6283
# Task:NcmFitMC, current time:        Wed Apr 06 2022, 00:36:27
# Task:NcmFitMC, estimated to end at: Wed Apr 06 2022, 00:42:43 +/- 00:02:23.6283
#  Elapsed time: 00 days, 00:01:14.2970140
#  iteration            [000073]
#  function evaluations [000075]
#  gradient evaluations [000000]
#  degrees of freedom   [011949]
#  m2lnL     =  1.43726352423712e-07 (  1.4372635e-07 )
#  Fit parameters:
#     0.259985135944353   -0.996799247761764    3.18422097243301    
# NcmMSet

#  Elapsed time: 00 days, 00:01:11.7408420
#  iteration            [000068]
#  function evaluations [000070]
#  gradient evaluations [000000]
#  degrees of freedom   [012295]
#  m2lnL     =  3.40002492676961e-06 (  3.4000249e-06 )
#  Fit parameters:
#     0.260816732771312   -0.996929336396184    3.20127317566854    
# NcmMSetCatalog: Current mean:   9.4045e-07   0.26036     -0.9958       3.1845     
# NcmMSetCatalog: Current msd:    2.8892e-07   0.00045838   0.00040939   0.0034268  
# NcmMSetCatalog: Current sd:     1.5013e-06   0.0023818    0.0021273    0.017806   
# NcmMSetCatalog: Current var:    2.2539e-12   5.673e-06    4.5252e-06   0.00031706 
# NcmMSetCatalog: Current tau:    1            1            1            1          
# Task:NcmFitMC, completed: 27 of 50, elapsed time: 00:05:07.3278
# Task:NcmFitMC, mean time: 00:00:11.3755 +/- 00:00:03.3517
# Task:NcmFitMC, time left: 00:04:21.6355 +/- 00:01:17.0893
# Task:NcmFitMC, current time:        Wed Apr 06 2022, 00:37:43
# Task

# NcmMSetCatalog: Current tau:    1            1            1            1          
# Task:NcmFitMC, completed: 34 of 50, elapsed time: 00:06:24.2846
# Task:NcmFitMC, mean time: 00:00:11.2976 +/- 00:00:02.7200
# Task:NcmFitMC, time left: 00:03:00.7617 +/- 00:00:43.5204
# Task:NcmFitMC, current time:        Wed Apr 06 2022, 00:39:00
# Task:NcmFitMC, estimated to end at: Wed Apr 06 2022, 00:42:01 +/- 00:00:43.5204
#  Elapsed time: 00 days, 00:01:28.2430650
#  iteration            [000076]
#  function evaluations [000078]
#  gradient evaluations [000000]
#  degrees of freedom   [012093]
#  m2lnL     =  5.94679967769185e-08 (  5.9467997e-08 )
#  Fit parameters:
#     0.259915135285401   -0.996421282147917    3.16870029750018    
# NcmMSetCatalog: Current mean:   1.2408e-06   0.26036     -0.996        3.1839     
# NcmMSetCatalog: Current msd:    2.8789e-07   0.00035292   0.00032337   0.0030802  
# NcmMSetCatalog: Current sd:     1.7032e-06   0.0020879    0.0019131    0.018222   
# NcmMSet

#     0.259957274056982   -0.996173394647777    3.18395793922348    
# NcmMSetCatalog: Current mean:   1.4475e-06   0.26072     -0.99607      3.1821     
# NcmMSetCatalog: Current msd:    3.3234e-07   0.00038359   0.00027425   0.0029233  
# NcmMSetCatalog: Current sd:     2.1538e-06   0.0024859    0.0017773    0.018945   
# NcmMSetCatalog: Current var:    4.6389e-12   6.1798e-06   3.159e-06    0.00035892 
# NcmMSetCatalog: Current tau:    1            1            1            1          
# Task:NcmFitMC, completed: 42 of 50, elapsed time: 00:07:48.5645
# Task:NcmFitMC, mean time: 00:00:11.1525 +/- 00:00:02.2387
# Task:NcmFitMC, time left: 00:01:29.2198 +/- 00:00:17.9092
# Task:NcmFitMC, current time:        Wed Apr 06 2022, 00:40:25
# Task:NcmFitMC, estimated to end at: Wed Apr 06 2022, 00:41:54 +/- 00:00:17.9092
#  Elapsed time: 00 days, 00:01:12.3287630
#  iteration            [000066]
#  function evaluations [000068]
#  gradient evaluations [000000]
#  degrees of freedom   [012126]

# Task:NcmFitMC, current time:        Wed Apr 06 2022, 00:41:07
# Task:NcmFitMC, estimated to end at: Wed Apr 06 2022, 00:41:17 +/- 00:00:01.9547
#  Elapsed time: 00 days, 00:00:41.4604780
#  iteration            [000075]
#  function evaluations [000077]
#  gradient evaluations [000000]
#  degrees of freedom   [012165]
#  m2lnL     =  6.40478530190836e-07 (  6.4047853e-07 )
#  Fit parameters:
#     0.261037501131373   -0.998400840689055    3.11254324069331    
# NcmMSetCatalog: Current mean:   1.4607e-06   0.26079     -0.99617      3.1792     
# NcmMSetCatalog: Current msd:    2.8721e-07   0.0003446    0.00025795   0.0031553  
# NcmMSetCatalog: Current sd:     2.0309e-06   0.0024367    0.001824     0.022311   
# NcmMSetCatalog: Current var:    4.1246e-12   5.9373e-06   3.327e-06    0.00049778 
# NcmMSetCatalog: Current tau:    1            1            1            1          
# Task:NcmFitMC, completed: 50 of 50, elapsed time: 00:08:32.1056
# Task:NcmFitMC, mean time: 00:00:10.2411 +/